In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/madhusudantamrakar/.wdm/drivers/chromedriver/mac64/92.0.4515.43/chromedriver] found in cache


# Visit the NASA mars news site

In [3]:
# Visit the Mars news site
# URL of the page to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading thre page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Create a Beautiful Soup object; parse with 'html.parser'
# Convert the browser html to a soup object
html = browser.html
news_soup = bs(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Moves Forward With Campaign to Return Mars Samples to Earth</div>

In [5]:
# Use the parent element to find the News Title
news_title = slide_elem.find('div', class_='content_title').get_text()

# Display the title output
print(f' news_title = {news_title}')

# Use the parent element to find the Paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

# Display the paragraph output
print(f' news_p = {news_p}')

 news_title = NASA Moves Forward With Campaign to Return Mars Samples to Earth
 news_p = During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.


# JPL Space Images - Featured Image

In [6]:
# Visit the Featured Space Image site
# URL of the page to be scraped
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [7]:
# Find and click the 'Full Image' button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [8]:
# Parse the resulting html with soup
html = browser.html
img_soup = bs(html, 'html.parser')

In [9]:
 # find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [10]:
# User the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'

# Display the absolute url
print(f'featured_image_url = {img_url}')

featured_image_url = https://spaceimages-mars.com/image/featured/mars3.jpg


# Mars Facts

In [11]:
# Use 'pd.read_html' to pull the data from the Mars-Earth Comparison section
# Mars Facts url for scraping
url = 'https://galaxyfacts-mars.com/'

df = pd.read_html(url)[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [12]:
df.columns = ['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [13]:
# Use Pandas to convert the data to a HTML table string.
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# Mars Hemispheres

In [14]:
# Url to Astrogeology site
url = 'https://marshemispheres.com/'

browser.visit(url)

In [15]:
# Create a list to hold the images and titles
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css('a.product-item img')

# Next, loop through those links, click the link, find the samaple anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css('a.product-item img')[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()

In [16]:
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [17]:
browser.quit()